<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/Atividade3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 3



Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [1]:
using Pkg
Pkg.activate("pastesian")
Pkg.add("JuMP")
Pkg.add("HiGHS")

  Activating new project at `/content/pastesian`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed CodecBzip2 ───────── v0.8.5
   Installed StructTypes ──────── v1.11.0
   Installed JSON3 ────────────── v1.14.3
   Installed BenchmarkTools ───── v1.6.3
   Installed ForwardDiff ──────── v1.3.0
   Installed MutableArithmetics ─ v1.6.7
   Installed MathOptInterface ─── v1.46.0
   Installed JuMP ─────────────── v1.29.3
    Updating `/content/pastesian/Project.toml`
  [4076af6c] + JuMP v1.29.3
    Updating `/content/pastesian/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [523fee87] + CodecBzip2 v0.8.5
  [944b1d66] + CodecZlib v0.7.8
  [bbf7d656] + CommonSubexpressions v0.3.1
  [34da2185] + Compat v4.18.1
  [864edb3b] + DataStructures v0.19.3
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + DiffRules v1.15.1
  [ffbed154] + DocStringExtensions v0.9.5
  [f6369f11] + ForwardDiff v1.3.0
  [92d709cd] + IrrationalConstants v0.2.6
  [69

In [2]:
using JuMP, HiGHS, LinearAlgebra, Printf

## Problema

A Pastesian é uma fábrica de massas familiar que está planejando a produção de lasanhas para os próximos 4 meses. Além do sabor tradicional, ela decidiu lançar um novo sabor de lasanha para esta temporada. A empresa planeja suas operações ao longo de 4 meses. Para o sabor tradicional, a demanda para os primeiros 3 meses é considerada conhecida com base em dados históricos: 200, 350 e 150 unidades, respectivamente.

A demanda do 4° mês para o sabor tradicional é mais incerta, pois coincide com a alta temporada na região e historicamente apresenta grande variabilidade. Para lidar com essa incerteza, Ricardo, o dono da Pastesian, trabalha com três cenários possíveis para a demanda do sabor tradicional no último mês, dados por:

- Cenário 1: 220 unidades  
- Cenário 2: 250 unidades  
- Cenário 3: 300 unidades  

Assume-se que esses cenários são equiprováveis, e o modelo de produção deve ser capaz de acomodar essas variações sem incorrer em custos excessivos de produção, estocagem ou falta de produto.

Paralelamente, o novo sabor de lasanha também precisa ser planejado. Estima-se que a demanda desse novo produto seja de 30, 70 e 140 unidades nos meses 1, 2 e 3, respectivamente. Para o 4° mês, a demanda pode assumir os seguintes valores, com as probabilidades esperadas indicadas a seguir:

| Demanda (unidades) | Probabilidade |
|--------------------|---------------|
| 200                | 0,3           |
| 240                | 0,5           |
| 300                | 0,2           |

Além disso, Ricardo pretende expandir gradualmente a infraestrutura de armazenamento ao longo da temporada. Dessa forma, a capacidade total de estoque da fábrica (somando lasanha tradicional e nova) em cada mês é limitada por:

| Mês | Capacidade de estoque (unidades) |
|-----|----------------------------------|
| 1   | 200                              |
| 2   | 220                              |
| 3   | 230                              |
| 4   | 250                              |

Para garantir o compromisso com os clientes, estabelecemos uma penalidade para demandas não atendidas. O custo estimado por unidade de demanda perdida (custo de falta) foi definido como superior ao custo de produção, incentivando o atendimento integral dos pedidos. Estima-se uma penalidade de [INSERIR VALOR AQUI] por unidade não entregue para qualquer sabor.

Além disso, a fábrica possui limitações físicas e operacionais para a nova linha de produção. A capacidade máxima absoluta é de 400 lasanhas por mês. A operação preferencial é de até 300 unidades mensais. Para volumes de produção entre 300 e 400 unidades, é necessário acionar equipamentos extras e turnos adicionais, o que incorre em um custo marginal adicional de R$ 0,35 por lasanha excedente (sobre o volume que ultrapassar 300).

O objetivo é determinar, para cada mês e para cada sabor, quanto produzir e quanto manter em estoque, de forma a minimizar o custo total de operação (produção, estocagem e eventuais faltas), respeitando as limitações impostas.

## Parâmetros

In [3]:
# Períodos (mês)
T = 1:4;
# Custo (R$/unidade) de produção em cada mês
cp = [5.5, 7.2, 8.8, 10.9];
# Custo (R$/unidade) de manter as lasanhas em estoque, de um mês para o seguinte
ce = [1.3, 1.95, 2.2];
# Capacidade total de estoque em cada mês (para as duas lasanhas somadas)
E = [200, 220, 230, 250];

# Modelo para lasanha original

**Parâmetros**

In [4]:
# Demanda certa nos meses 1–3 (lasanha original)
D_o = [200, 350, 150];
# Lasanhas originais (unidade) em estoque inicialmente
e0_o = 50;

# Cenários para o mês 4
C_o = 1:3;
# Demanda incerta no mês 4
D4_o = [220, 250, 300]
# Probabilidade de cada cenário Original = 1/3
Po = 1.0 / length(D4_o)
# Penalidade por demanda não atendida no mês 4
pp_o = 3 * cp[4]

32.7

**Modelo**

Esta função constrói a estrutura da Lasanha Original, contendo apenas os elementos fixos e comuns aos problemas subsequentes. Ela representa a base necessária para o planejamento dos meses com demanda certa (Mês 1, 2 e 3) e será utilizada tanto na formulação do modelo determinístico quanto no modelo estocástico.

Observe que a função objetivo e o tratamento da incerteza do mês 4 não são definidos aqui.

O foco desta função base está apenas nas restrições estruturais da lasanha original para os primeiros três meses:

* **Estoque inicial:** Fixa o estoque da lasanha original no início da temporada: `e_o[1] = e0_o`;
* **Balanço de estoque:** Define a relação de fluxo entre produção, demanda e estoque ao longo dos três primeiros meses: `e_o[t+1] = e_o[t] + l_o[t] - D_orig[t]` para $t \in \{1, 2, 3\}$.

As variáveis de produção ($\mathbf{l}_o[4]$) e estoque ($\mathbf{e}_o[4]$) do Mês 4 são criadas neste ponto, mas suas restrições serão adicionadas na próxima etapa de cada modelo.

In [5]:
function build_base_original(model, D_o)

    # === VARIÁVEIS ===
    @variable(model, l_o[1:4] >= 0)      # Produção da lasanha original
    @variable(model, e_o[0:4] >= 0)      # Estoque da lasanha original

    # === RESTRIÇÕES (Meses 1, 2, 3) ===

    # Estoque inicial
    @constraint(model, e_o[0] == e0_o)

    # Balanço meses 1-3: estoque_final[t] = estoque_final[t-1] + produção[t] - demanda[t]
    @constraint(model, BAL_o[t in 1:3],
        e_o[t] == e_o[t-1] + l_o[t] - D_o[t]
    )

    return model, l_o, e_o
end

build_base_original (generic function with 1 method)

# Modelo para lasanha nova

**Parâmetros**

In [6]:
# Demanda certa nos meses 1–3 (lasanha nova)
D_n = [30, 70, 140];

# Cenários para o mês 4
C_n = 1:3;
# Demanda incerta no mês 4
D4_n = [200, 240, 300];
# Probabilidades
P4_n = [0.3, 0.5, 0.2];
# Estoque inicial
e0_n = 0;

# Custo adicional ($/unidade) para produção acima do limite suave
custo_extra = 0.35;
# Limite de produção preferencial (sem custo extra)
limite_suave = 300;
# Capacidade máxima absoluta de produção da lasanha nova
limite_maximo = 400;

**Modelo**

De forma análoga ao produto original, construímos a estrutura básica para o novo sabor de lasanha. A principal distinção reside na condição inicial: por tratar-se de um lançamento, o estoque inicial é nulo ($e0\_n = 0$).

Novamente, esta função foca na dinâmica apenas dos três primeiros meses, visto que a única diferença entre os modelos determinístico e estocásticos para a lasanha nova está no 4° mês.

In [7]:
function build_base_nova(model, D_n)
    # === VARIÁVEIS ===
    @variable(model, l_n[1:4] >= 0)      # Produção
    @variable(model, e_n[0:4] >= 0)      # Estoque (0=inicial, 1-4=final de cada mês)

    # Produção excedente (acima de 300)
    # Ela será usada apenas para calcular o custo extra na função objetivo
    @variable(model, l_n_over[1:4] >= 0)

    # === RESTRIÇÕES (Meses 1-3) ===

    # Estoque inicial zero
    @constraint(model, e_n[0] == e0_n)

    # Balanço meses 1-3
    @constraint(model, BAL_n[t in 1:3],
        e_n[t] == e_n[t-1] + l_n[t] - D_n[t]
    )

    # Teto Máximo de Produção
    @constraint(model, Cap_Max_n[t in 1:4],
        l_n[t] <= limite_maximo
    )

    # Definição do Excedente
    # Se l_n < 300, l_n_over será 0. Se l_n > 300, será a diferença
    @constraint(model, Def_Over_n[t in 1:4],
        l_n_over[t] >= l_n[t] - limite_suave
    )

    return model, l_n, e_n, l_n_over
end

build_base_nova (generic function with 1 method)

## Modelo determinístico

Nesta etapa, construímos um modelo unificado para o planejamento da produção. Devido à natureza distinta das incertezas, adotamos abordagens diferentes para cada produto no 4º mês:

- **Lasanha nova:** Como é um produto novo e a variabilidade é estimada, simplificamos o problema substituindo a demanda incerta do mês 4 pelo seu valor esperado (240 unidades). Isso transforma o problema em um modelo determinístico clássico.

- **Lasanha original:** Para o produto consolidado, mantemos os cenários de demanda para calcular o risco de falta de produto.

O modelo determinístico para o novo sabor de lasanha é construído utilizando a demanda média esperada para o mês 4. A demanda incerta do mês 4 é resolvida pelo seu valor médio esperado, `d4_mean_n = 240` unidades.

In [8]:
d4_mean_n = dot(D4_n, P4_n);
D_n_det = vcat(D_n, d4_mean_n)

4-element Vector{Float64}:
  30.0
  70.0
 140.0
 240.0

**Modelo**

O modelo é iniciado e as variáveis do sabor original são carregadas para os meses 1,2 e 3.

In [9]:
model_det = Model(HiGHS.Optimizer);
model_det, l_o, e_o = build_base_original(model_det, D_o);
model_det, l_n, e_n, l_n_over = build_base_nova(model_det, D_n);

Já para o 4° mês, as seguintes variáveis são introduzidas:

In [10]:
# Déficit por cenário (lasanha original)
@variable(model_det, dp_o[C_o] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{VariableRef}:
 dp_o[1]
 dp_o[2]
 dp_o[3]

In [11]:
# Balanço lasanha nova
@constraint(model_det, BAL_n_4,
    e_n[4] == e_n[3] + l_n[4] - d4_mean_n
)

# Atendimento lasanha original por cenário (permite déficit)
# Produção[4] + Estoque[3] >= Demanda[cenário] - Déficit[cenário]
@constraint(model_det, DEM_o[c in C_o],
    l_o[4] + e_o[3] >= D4_o[c] - dp_o[c]
)

# Balanço lasanha original
d4_mean_o = sum(Po * D4_o[c] for c in C_o)
@constraint(model_det, BAL_o_4,
    e_o[4] == e_o[3] + l_o[4] - d4_mean_o
)

BAL_o_4 : -l_o[4] - e_o[3] + e_o[4] = -256.66666666666663

Além disso, o armazenamento é compartilhada entre ambos os tipos de lasanhas. Logo precisamos introduzir a seguinte restrição:

In [12]:
# Capacidade no final de cada mês (estoque somado)
@constraint(model_det, CE[t in 1:4],
    e_o[t] + e_n[t] <= E[t]
)

4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 CE[1] : e_o[1] + e_n[1] ≤ 200
 CE[2] : e_o[2] + e_n[2] ≤ 220
 CE[3] : e_o[3] + e_n[3] ≤ 230
 CE[4] : e_o[4] + e_n[4] ≤ 250

A função objetivo é a soma dos custos determinísticos do novo sabor com o custo esperado dos componentes estocásticos do sabor original.

In [13]:
custo_1_3 = (
    sum(cp[t] * (l_o[t] + l_n[t]) for t in 1:3)  # Produção
    + sum(ce[t] * (e_o[t] + e_n[t]) for t in 1:3)  # Estoque
    + sum(custo_extra * l_n_over[t] for t in 1:3) # Custo de sobrecarga lasanha nova
)

# Custo esperado mês 4
custo_mes_4 = (
    cp[4] * l_o[4]  # Produção original
    + cp[4] * l_n[4]  # Produção nova
    + custo_extra * l_n_over[4] # Custo de sobrecarga lasanha nova
    + sum(Po * pp_o * dp_o[c] for c in C_o)  # Penalidade esperada
)

@objective(model_det, Min, custo_1_3 + custo_mes_4)

5.5 l_o[1] + 5.5 l_n[1] + 7.2 l_o[2] + 7.2 l_n[2] + 8.8 l_o[3] + 8.8 l_n[3] + 1.3 e_o[1] + 1.3 e_n[1] + 1.95 e_o[2] + 1.95 e_n[2] + 2.2 e_o[3] + 2.2 e_n[3] + 0.35 l_n_over[1] + 0.35 l_n_over[2] + 0.35 l_n_over[3] + 10.9 l_o[4] + 10.9 l_n[4] + 0.35 l_n_over[4] + 10.9 dp_o[1] + 10.9 dp_o[2] + 10.9 dp_o[3]

In [14]:
optimize!(model_det)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 25 rows; 25 cols; 55 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [3e-01, 1e+01]
  Bound   [0e+00, 0e+00]
  RHS     [3e+01, 4e+02]
Presolving model
17 rows, 20 cols, 42 nonzeros  0s
Dependent equations search running on 6 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
16 rows, 20 cols, 41 nonzeros  0s
Presolve reductions: rows 16(-9); columns 20(-5); nonzeros 41(-14) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     9.9000246219e+02 Pr: 9(1976.67) 0s
         12     1.2372000000e+04 Pr: 0(0) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 12
Objective value     :  1.2372000000e+04
P-D objective error :  0.0000000000e+00


**Resultados**

In [15]:
println("Status: ", termination_status(model_det))
println("Custo Total Ótimo: R\$ ", round(objective_value(model_det), digits=2))

println("\n--- LASANHA ORIGINAL ---")
println("Produção:           ", round.(value.(l_o), digits=1))
println("Estoque (final):    ", round.(value.(e_o), digits=1))
println("Déficit mês 4:      ", round.(value.(dp_o), digits=1))

println("\n--- LASANHA NOVA ---")
println("Produção Total:     ", round.(value.(l_n), digits=1))
println("Prod. Excedente (>300): ", round.(value.(l_n_over), digits=1))
println("Estoque (final):    ", round.(value.(e_n), digits=1))

println("\n--- UTILIZAÇÃO DE CAPACIDADE (ARMAZÉM) ---")
for t in 1:4
    usado = value(e_o[t]) + value(e_n[t])
    pct = round(100 * usado / E[t], digits=1)
    println("Mês $t: $(round(usado, digits=1))/$(E[t]) unidades ($pct%)")
end

Status: OPTIMAL
Custo Total Ótimo: R$ 12372.0

--- LASANHA ORIGINAL ---
Produção:           [280.0, 220.0, 150.0, 256.7]
Estoque (final):    1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:4
And data, a 5-element Vector{Float64}:
  50.0
 130.0
   0.0
   0.0
   0.0
Déficit mês 4:      1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
  0.0
  0.0
 43.3

--- LASANHA NOVA ---
Produção Total:     [100.0, 0.0, 140.0, 240.0]
Prod. Excedente (>300): [0.0, 0.0, 0.0, 0.0]
Estoque (final):    1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:4
And data, a 5-element Vector{Float64}:
 -0.0
 70.0
  0.0
  0.0
  0.0

--- UTILIZAÇÃO DE CAPACIDADE (ARMAZÉM) ---
Mês 1: 200.0/200 unidades (100.0%)
Mês 2: 0.0/220 unidades (0.0%)
Mês 3: 0.0/230 unidades (0.0%)
Mês 4: 0.0/250 unidades (0.0%)


## 1.5. Análise de sensibilidade


Agora que encontramos o plano de produção ideal, precisamos entender os limites da fábrica. Não basta saber quanto produzir; precisamos saber onde estão os problemas (gargalos) e onde podemos economizar mais. Vamos analisar quatro pontos principais.

In [16]:
report = lp_sensitivity_report(model_det);

**Capacidade de armazenamento**

Aqui verificamos se o armazém está lotado em algum mês. Se o estoque estiver no limite, o modelo calcula o "preço sombra". Esse valor nos diz exatamente quanto dinheiro economizaríamos se tivéssemos espaço para guardar apenas mais uma lasanha. Se houver espaço sobrando (folga), expandir o armazém não vale a pena agora.

In [17]:
for t in 1:4
    constraint_ref = model_det[:CE][t]
    dual_val = dual(constraint_ref) # Preço sombra
    usado = value(e_o[t]) + value(e_n[t])
    folga = E[t] - usado

    println("\nMês $t (Capacidade: $(E[t]) unidades)")
    println("   Utilização: $(round(usado, digits=1)) unidades")

    if dual_val < -0.001
        val_abs = abs(dual_val)
        println("GARGALO DETECTADO!")
        println("- Custo marginal: A falta de espaço custa R\$ $(round(val_abs, digits=2)) por unidade")
        println("- Interpretação: Aumentar o armazém em 1 unidade reduziria o custo total em R\$ $(round(val_abs, digits=2))")
    else
        println("Folga: $(round(folga, digits=1)) unidades livres")
        println("Capacidade ociosa - expandir o armazém não reduz custos agora")
    end
end



Mês 1 (Capacidade: 200 unidades)
   Utilização: 200.0 unidades
GARGALO DETECTADO!
- Custo marginal: A falta de espaço custa R$ 0.4 por unidade
- Interpretação: Aumentar o armazém em 1 unidade reduziria o custo total em R$ 0.4

Mês 2 (Capacidade: 220 unidades)
   Utilização: 0.0 unidades
Folga: 220.0 unidades livres
Capacidade ociosa - expandir o armazém não reduz custos agora

Mês 3 (Capacidade: 230 unidades)
   Utilização: 0.0 unidades
Folga: 230.0 unidades livres
Capacidade ociosa - expandir o armazém não reduz custos agora

Mês 4 (Capacidade: 250 unidades)
   Utilização: 0.0 unidades
Folga: 250.0 unidades livres
Capacidade ociosa - expandir o armazém não reduz custos agora


**Custo de produção**

Os preços dos ingredientes podem variar. Nesta etapa, calculamos a margem de segurança dos custos atuais. Queremos saber: "Até que preço posso continuar produzindo essa mesma quantidade sem ter prejuízo?".

In [18]:
for t in 1:3
    println("\nMês $t (Custo atual: R\$ $(cp[t])/unidade)")

    # Lasanha Original
    prod_o = value(l_o[t])
    println("  Original - Produção: $(round(prod_o, digits=1))")

    try
        range_o = report[l_o[t]]
        subir = range_o[2] < 1e10 ? "R\$ $(round(cp[t] + range_o[2], digits=2))" : "+Infinito"
        cair  = range_o[1] > -1e10 ? "R\$ $(round(cp[t] + range_o[1], digits=2))" : "-Infinito"
        println("     Intervalo de estabilidade: de $cair até $subir")
    catch
        println("     (Sensibilidade não calculada para variação de base)")
    end

    # Lasanha Nova
    prod_n = value(l_n[t])
    println("  Nova - Produção: $(round(prod_n, digits=1))")

    try
        range_n = report[l_n[t]]
        subir_n = range_n[2] < 1e10 ? "R\$ $(round(cp[t] + range_n[2], digits=2))" : "+Infinito"
        cair_n  = range_n[1] > -1e10 ? "R\$ $(round(cp[t] + range_n[1], digits=2))" : "-Infinito"
        println("    Intervalo de estabilidade: de $cair_n até $subir_n")
    catch
        println("    (Sensibilidade não calculada)")
    end
end



Mês 1 (Custo atual: R$ 5.5/unidade)
  Original - Produção: 280.0
     Intervalo de estabilidade: de R$ 5.5 até R$ 5.85
  Nova - Produção: 100.0
    Intervalo de estabilidade: de R$ 5.15 até R$ 5.5

Mês 2 (Custo atual: R$ 7.2/unidade)
  Original - Produção: 220.0
     Intervalo de estabilidade: de R$ 6.85 até R$ 7.2
  Nova - Produção: 0.0
    Intervalo de estabilidade: de R$ 7.2 até +Infinito

Mês 3 (Custo atual: R$ 8.8/unidade)
  Original - Produção: 150.0
     Intervalo de estabilidade: de R$ 8.7 até R$ 9.15
  Nova - Produção: 140.0
    Intervalo de estabilidade: de R$ 8.7 até R$ 9.15


**Gargalos da lasanha nova**

Analisamos a intensidade da produção do novo sabor. O objetivo é ver se estamos operando na "zona de conforto" (até 300 unidades) ou se estamos pagando taxas extras para produzir mais. Além disso, verificamos se atingimos o teto físico da fábrica (400 unidades), o que poderia significar que estamos perdendo vendas por falta de maquinário.

In [19]:
for t in 1:4
    prod = value(l_n[t])
    over = value(l_n_over[t])

    println("\nMês $t - Produção: $(round(prod, digits=1))")

    # Como definimos @constraint(..., l_n <= 400), em minimização o dual será negativo se ativa
    dual_cap = dual(model_det[:Cap_Max_n][t])

    if abs(prod - 400) < 0.1
        println(" LIMITE RÍGIDO ATINGIDO (400 un)")
        println("   A fábrica está operando no teto físico.")
        if abs(dual_cap) > 0.001
            println("   Preço Sombra: Expandir a fábrica em 1 unidade economizaria R\$ $(round(abs(dual_cap), digits=2)).")
        end
    elseif prod > 300.1
        println(" OPERAÇÃO EM SOBRECARGA (>300 un)")
        println("   Estamos produzindo $(round(over, digits=1)) unidades com custo extra.")
        custo_add = over * custo_extra
        println("   Custo adicional de operação: R\$ $(round(custo_add, digits=2))")
    else
        println("  Operação Normal (<300 un)")
        println("   Operando dentro da capacidade preferencial.")
    end
end


Mês 1 - Produção: 100.0
  Operação Normal (<300 un)
   Operando dentro da capacidade preferencial.

Mês 2 - Produção: 0.0
  Operação Normal (<300 un)
   Operando dentro da capacidade preferencial.

Mês 3 - Produção: 140.0
  Operação Normal (<300 un)
   Operando dentro da capacidade preferencial.

Mês 4 - Produção: 240.0
  Operação Normal (<300 un)
   Operando dentro da capacidade preferencial.


## Modelo estocástico

Como a incerteza no Mês 4 é dada por cenários de demanda independentes para a lasanha original ($\mathbf{C}_o = 3$ cenários) e a lasanha nova ($\mathbf{C}_n = 3$ cenários), o conjunto total de cenários estocásticos é:
$$
  S = \mathbf{C}_o \times \mathbf{C}_n \implies |S| = 3 \times 3 = 9 \text{ cenários.}
$$

Cada cenário $s \in S$ tem uma probabilidade $\mathbf{P}[s] = \mathbf{P}[s_o] \cdot \mathbf{P}[s_n]$.

In [20]:
scenarios = [(co, cn) for co in C_o, cn in 1:length(D4_n)]
S = vec(scenarios)
P_scenario = Dict(s => Po * P4_n[s[2]] for s in S)

println("\nTotal de cenários: $(length(S))")
println("Soma das probabilidades: $(sum(values(P_scenario)))")


Total de cenários: 9
Soma das probabilidades: 0.9999999999999999


**Modelo**

In [35]:
model_esto = Model(HiGHS.Optimizer);
model_esto, l_o_base, e_o_base = build_base_original(model_esto, D_o);
model_esto, l_n_base, e_n_base, l_n_over_base = build_base_nova(model_esto, D_n);

l_o_1 = l_o_base; e_o_1 = e_o_base
l_n_1 = l_n_base; e_n_1 = e_n_base
l_n_over_1 = l_n_over_base;

In [36]:
# Capacidade de armazenamento (Meses 1-3)
@constraint(model_esto, CE_1[t in 1:3],
    e_o_base[t] + e_n_base[t] <= E[t]
);

In [37]:
# Variáveis dependentes do cenário (indexadas por s em S)
@variable(model_esto, l_o_2[S] >= 0)    # Produção Original Mês 4
@variable(model_esto, l_n_2[S] >= 0)    # Produção Nova Mês 4
@variable(model_esto, e_o_2[S] >= 0)    # Estoque Final Original
@variable(model_esto, e_n_2[S] >= 0)    # Estoque Final Nova
@variable(model_esto, dp_o_2[S] >= 0)   # Déficit Original
@variable(model_esto, dp_n_2[S] >= 0)   # Déficit Nova

# Excedente de produção no Mês 4
@variable(model_esto, l_n_over_2[S] >= 0);

In [38]:
# === RESTRIÇÕES - SEGUNDO ESTÁGIO (por cenário) ===
for s in S
    # CORREÇÃO: s já é a tupla (idx_o, idx_n), basta desempacotar diretamente
    idx_o, idx_n = s

    demanda_o = D4_o[idx_o]
    demanda_n = D4_n[idx_n]

    # Balanço de massa - Original
    # Nota: s é usado como índice da variável porque definimos @variable(..., x[S])
    @constraint(model_esto,
        l_o_2[s] + e_o_1[3] + dp_o_2[s] == demanda_o + e_o_2[s]
    )

    # Balanço de massa - Nova
    @constraint(model_esto,
        l_n_2[s] + e_n_1[3] + dp_n_2[s] == demanda_n + e_n_2[s]
    )

    # Capacidade de Estoque Mês 4
    @constraint(model_esto,
        e_o_2[s] + e_n_2[s] <= E[4]
    )

    # REGRAS DA LASANHA NOVA (Mês 4)
    # Capacidade máxima física (400)
    @constraint(model_esto, l_n_2[s] <= limite_maximo)

    # Definição do excedente (>300) para cobrar taxa extra
    @constraint(model_esto, l_n_over_2[s] >= l_n_2[s] - limite_suave)
end

In [41]:
# Loop de restrições por cenário
for s in S
    # CORREÇÃO: s já é a tupla (idx_o, idx_n), não precisa buscar em scenarios[]
    idx_o, idx_n = s

    demanda_o = D4_o[idx_o]
    demanda_n = D4_n[idx_n]

    # 1. Balanço de Massa (Conecta com estoque do mês 3 da base)
    # Original: Prod + Estq_Ant + Déficit = Demanda + Estq_Final
    @constraint(model_esto,
        l_o_2[s] + e_o_1[3] + dp_o_2[s] == demanda_o + e_o_2[s]
    )

    # Nova: Prod + Estq_Ant + Déficit = Demanda + Estq_Final
    @constraint(model_esto,
        l_n_2[s] + e_n_1[3] + dp_n_2[s] == demanda_n + e_n_2[s]
    )

    # 2. Capacidade de Estoque Mês 4
    @constraint(model_esto,
        e_o_2[s] + e_n_2[s] <= E[4]
    )

    # 3. REGRAS DA LASANHA NOVA NO MÊS 4
    # Capacidade Máxima Física (400)
    @constraint(model_esto, l_n_2[s] <= limite_maximo)

    # Definição do Excedente (>300) para cobrar taxa extra
    @constraint(model_esto, l_n_over_2[s] >= l_n_2[s] - limite_suave)
end

In [42]:

# === FUNÇÃO OBJETIVO ===

# Custo Determinístico (Meses 1-3)
custo_1_estagio = (
    sum(cp[t] * (l_o_1[t] + l_n_1[t]) for t in 1:3)
    + sum(ce[t] * (e_o_1[t] + e_n_1[t]) for t in 1:3)
    + sum(custo_extra * l_n_over_1[t] for t in 1:3) # Custo extra Nova (1-3)
)

# Custo Esperado (Mês 4)
custo_2_estagio = sum(
    P_scenario[s] * (
        cp[4] * (l_o_2[s] + l_n_2[s])              # Produção Base
        + custo_extra * l_n_over_2[s]              # Custo Extra Nova (>300)
        + pp_o * dp_o_2[s]                         # Penalidade Original
        + pp_o * dp_n_2[s]                         # Penalidade Nova (Assumindo mesmo valor)
    )
    for s in S
)

@objective(model_esto, Min, custo_1_estagio + custo_2_estagio)

# === RESOLVER ===
optimize!(model_esto)

println("Status: ", termination_status(model_esto))
println("Custo Total Esperado: R\$ ", round(objective_value(model_esto), digits=2))

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 109 rows; 85 cols; 272 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [2e-02, 9e+00]
  Bound   [0e+00, 0e+00]
  RHS     [3e+01, 4e+02]
Presolving model
82 rows, 75 cols, 239 nonzeros  0s
45 rows, 66 cols, 121 nonzeros  0s
Dependent equations search running on 22 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
45 rows, 66 cols, 121 nonzeros  0s
Presolve reductions: rows 45(-64); columns 66(-19); nonzeros 121(-151) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     9.9000195352e+02 Pr: 22(5240) 0s
         28     1.1899666667e+04 Pr: 0(0) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 28
Objective value     :  1.1899666667e+0

In [43]:
println("\n" * "="^60)
println("RESULTADOS - MODELO ESTOCÁSTICO")
println("="^60)

println("\nStatus: ", termination_status(model_esto))
println("Custo Total Ótimo: R\$ ", round(objective_value(model_esto), digits=2))

println("\n--- DECISÕES DE PRIMEIRO ESTÁGIO (Meses 1-3) ---")
println("\nLasanha Original:")
println("  Produção:        ", round.(value.(l_o_1[1:3]), digits=1))
println("  Estoque (final): ", round.(value.(e_o_1[0:3]), digits=1))

println("\nLasanha Nova:")
println("  Produção:        ", round.(value.(l_n_1[1:3]), digits=1))
println("  Estoque (final): ", round.(value.(e_n_1[0:3]), digits=1))

println("\n--- DECISÕES DE SEGUNDO ESTÁGIO (Mês 4) ---")
println("\nCen | D_orig | D_nova | Prod_o | Prod_n | Def_o | Def_n | Prob  ")
println(repeat("-", 70))

for s in S
    co, cn = s
    println(
        @sprintf("%d,%d", co, cn), " | ",
        @sprintf("%6d", D4_o[co]), " | ",
        @sprintf("%6d", D4_n[cn]), " | ",
        @sprintf("%6.1f", value(l_o_2[s])), " | ",
        @sprintf("%6.1f", value(l_n_2[s])), " | ",
        @sprintf("%5.1f", value(dp_o_2[s])), " | ",
        @sprintf("%5.1f", value(dp_n_2[s])), " | ",
        @sprintf("%.3f", P_scenario[s])
    )
end

# Valores esperados
println("\n--- VALORES ESPERADOS (Mês 4) ---")
prod_o_esp = sum(P_scenario[s] * value(l_o_2[s]) for s in S)
prod_n_esp = sum(P_scenario[s] * value(l_n_2[s]) for s in S)
def_o_esp = sum(P_scenario[s] * value(dp_o_2[s]) for s in S)
def_n_esp = sum(P_scenario[s] * value(dp_n_2[s]) for s in S)

println("Produção original: $(round(prod_o_esp, digits=1)) unidades")
println("Produção nova:     $(round(prod_n_esp, digits=1)) unidades")
println("Déficit original:  $(round(def_o_esp, digits=2)) unidades")
println("Déficit nova:      $(round(def_n_esp, digits=2)) unidades")


RESULTADOS - MODELO ESTOCÁSTICO

Status: OPTIMAL
Custo Total Ótimo: R$ 11899.67

--- DECISÕES DE PRIMEIRO ESTÁGIO (Meses 1-3) ---

Lasanha Original:
  Produção:        [350.0, 150.0, 150.0]
  Estoque (final): 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [0, 1, 2, 3]
And data, a 4-element Vector{Float64}:
  50.0
 200.0
   0.0
   0.0

Lasanha Nova:
  Produção:        [30.0, 70.0, 140.0]
  Estoque (final): 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [0, 1, 2, 3]
And data, a 4-element Vector{Float64}:
 -0.0
  0.0
  0.0
  0.0

--- DECISÕES DE SEGUNDO ESTÁGIO (Mês 4) ---

Cen | D_orig | D_nova | Prod_o | Prod_n | Def_o | Def_n | Prob  
----------------------------------------------------------------------
1,1 |    220 |    200 |  220.0 |  200.0 |   0.0 |   0.0 | 0.100
2,1 |    250 |    200 |  250.0 |  200.0 |   0.0 |   0.0 | 0.100
3,1 |    300 |    200 |  300.0 |  200.0 |   0.0 |   0.0 | 0.100
1,2 |    220 |    240 |  220.0 